In [0]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import pickle
import torch 
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import random
import matplotlib.pyplot as plt
import re
import torch.nn.functional as F
import matplotlib.pyplot as plt
vgg_path = "drive/My Drive/common/cnn_features"
corpus_path = "drive/My Drive/common/video_corpus.csv"
model_save_path1 = "drive/My Drive/main/model_loss1"
yolo_path = "drive/My Drive/common/object_features"
wordtoindex_path = "drive/My Drive/common/wordtoindex.pickle"
indextoword_path = "drive/My Drive/common/indextoword.pickle"
embedding_path = "drive/My Drive/common/embedding.npy"
train_names_path = "drive/My Drive/common/train_names.npy"
test_names_path = "drive/My Drive/common/test_names.npy"
valid_names_path = "drive/My Drive/common/valid_names.npy"
sentence_model_path = "drive/My Drive/common/new_sentence_model"
model_save_path2 = "drive/My Drive/main/model_loss2"
model_save_path3 = "drive/My Drive/main/model_loss3"
model_save_prob = "drive/My Drive/main/model_prob"
model_save_teacher = "drive/My Drive/main/teacher_model"
model_sent_teacher = "drive/My Drive/main/teacher_model_sent"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sentence_model_number = 500
max_caption_length = 25
dim_image = 4096
dim_hidden1 = 1000
dim_hidden2 = 1000
dim_input1 = 1000
dim_input2 = 1300
embedding_dim = 300
batch_size = 50
lr = 0.00005
sent_lr = .00001
#video_model_num = 190


In [0]:
def save_dict(path , dct):
    with open(path , "wb") as handle:
        pickle.dump(dct , handle , protocol = pickle.HIGHEST_PROTOCOL)
def load_dict(path):
    with open(path, "rb") as handle:
        b = pickle.load(handle)
    return b

In [0]:
embedding = np.load(embedding_path).astype(np.float32)
wordtoindex = load_dict(wordtoindex_path)
indextoword = load_dict(indextoword_path)
embedding = torch.tensor(embedding).to(device).double()

In [0]:
def clean(s):
    regex = re.compile('[^a-zA-Z" "]')
    s =  regex.sub("" , s).strip().lower()
    new_w = [wordtoindex["<bos>"]]
    for w in s.split(" "):
        if(w in wordtoindex):
            new_w.append(wordtoindex[w])
    new_w.append(wordtoindex["<eos>"])
    return new_w
        

In [0]:
def get_data(tp):
    video_data = pd.read_csv(corpus_path, sep=',')
    video_data = video_data[video_data['Language'] == 'English']
    video_data['video_name'] = video_data.apply(lambda row: row['VideoID']+'_'+str(int(row['Start']))+'_'+str(int(row['End']))+'.avi.npy', axis=1)
    video_data['video_path'] = video_data.apply(lambda row: row['VideoID']+'_'+str(int(row['Start']))+'_'+str(int(row['End']))+'.avi.npy', axis=1)
    video_data['yolo_path'] = video_data.apply(lambda row: row['VideoID']+'_'+str(int(row['Start']))+'_'+str(int(row['End']))+'.avi.npy', axis=1)
    video_data['yolo_path'] = video_data['yolo_path'].map(lambda x: os.path.join(yolo_path, x))
    video_data['video_path'] = video_data['video_path'].map(lambda x: os.path.join(vgg_path, x))
    if(tp == "train"):
        names = np.load(train_names_path , allow_pickle = True)

    elif(tp == "test"):
        names =  np.load(test_names_path , allow_pickle = True)
    elif(tp == "valid"):
        names = np.load(valid_names_path , allow_pickle = True)
    names = names.tolist()
    video_data = video_data[video_data['video_name'].map(lambda x: x in names)]
    video_data = video_data[video_data['Description'].map(lambda x: isinstance(x, str))]
    video_data["Description"] = video_data["Description"].map(lambda x : clean(x))
    video_data = video_data[video_data['Description'].map(lambda x: True if len(x) > 3 and len(x) < max_caption_length else False )]
    unique_filenames = sorted(video_data['video_path'].unique())
    data = video_data[video_data['video_path'].map(lambda x: x in unique_filenames)]
    print(len(data))
    return data.groupby(["video_path" , "yolo_path"])

In [0]:
def get_data_clean(tp):
    video_data = pd.read_csv(corpus_path, sep=',')
    video_data = video_data[video_data['Language'] == 'English']
    video_data = video_data[video_data['Source'] == 'clean']
    video_data['video_name'] = video_data.apply(lambda row: row['VideoID']+'_'+str(int(row['Start']))+'_'+str(int(row['End']))+'.avi.npy', axis=1)
    video_data['video_path'] = video_data.apply(lambda row: row['VideoID']+'_'+str(int(row['Start']))+'_'+str(int(row['End']))+'.avi.npy', axis=1)
    video_data['yolo_path'] = video_data.apply(lambda row: row['VideoID']+'_'+str(int(row['Start']))+'_'+str(int(row['End']))+'.avi.npy', axis=1)
    video_data['yolo_path'] = video_data['yolo_path'].map(lambda x: os.path.join(yolo_path, x))
    video_data['video_path'] = video_data['video_path'].map(lambda x: os.path.join(vgg_path, x))
    if(tp == "train"):
        names = np.load(train_names_path , allow_pickle = True)

    elif(tp == "test"):
        names =  np.load(test_names_path , allow_pickle = True)
    elif(tp == "valid"):
        names = np.load(valid_names_path , allow_pickle = True)
    names = names.tolist()
    video_data = video_data[video_data['video_name'].map(lambda x: x in names)]
    video_data = video_data[video_data['Description'].map(lambda x: isinstance(x, str))]
    video_data["Description"] = video_data["Description"].map(lambda x : clean(x))
    video_data = video_data[video_data['Description'].map(lambda x: True if len(x) > 3 and len(x) < max_caption_length else False )]
    unique_filenames = sorted(video_data['video_path'].unique())
    data = video_data[video_data['video_path'].map(lambda x: x in unique_filenames)]
    print(len(data))
    return data.groupby(["video_path" , "yolo_path"])

In [7]:
df = get_data("train")


49175


In [0]:
video_dict = {}
yolo_dict = {}

for g in df:
    video_dict[g[0][0]] = torch.tensor(np.load(g[0][0] , allow_pickle = False))
    yolo_dict[g[0][1]] = torch.tensor(np.load(g[0][1] ,  allow_pickle = False))


In [9]:
gru_inp = 1000
gru_hidden = 1000
gru_inp = 1000
gru_hidden = 1000
class sim(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb_gru = nn.Linear(embedding_dim , gru_hidden)
        self.gru = nn.GRU(gru_inp,  gru_hidden  , batch_first = True,bidirectional = True)
        self.embedding = nn.Embedding(len(wordtoindex), embedding_dim)
        self.embedding.weight = nn.Parameter(embedding)
        self.embedding.weight.requires_grad = False
        self.gru_last = nn.Linear(gru_hidden*2 , gru_hidden)
         
    def forward(self , X):
        emb = self.embedding(X)
        out = self.emb_gru(emb)
        out , _ = self.gru(out)
        out = out[: , -1, :]
        out = self.gru_last(out)
        return out
        

model = sim().to(device).double()
model_num = sentence_model_number
model.load_state_dict(torch.load(os.path.join(sentence_model_path , "model" + str(model_num) + ".pt") ))
#model  = model.eval()


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
def get_batch(df):
    batch = df.apply(lambda x : x.iloc[np.random.choice(len(x))])
    batch = batch.reset_index(drop = True)
    batch = batch.sample(batch_size)
    videos = list(map(lambda x : video_dict[x],   batch["video_path"].values))
    objects = list(map(lambda x :yolo_dict[x] ,   batch["yolo_path"].values))
    videos = pad_sequence(videos , batch_first = True)
    objects = pad_sequence(objects , batch_first = True)
    description = batch.Description.tolist()
    description = list(map(lambda x : torch.tensor(x) , description))
    
    description = pad_sequence(description , batch_first = True)
    pad = torch.zeros(description.size(0) , max_caption_length - description.size(1)).long()
    description = torch.cat((description , pad) , dim = 1)
    targets = description[: , 1:]
    feed  = description[: , :-1]
    
#    for v in videos:
#        print(v.shape)
#        break
    return videos.to(device).double() , objects.to(device).double() , feed.to(device)  ,  targets.to(device) 
    


    

In [0]:
dist = nn.PairwiseDistance(p=1 , eps = 1e-6)
def distance(dist_model, emb , batch_size):
    out = dist_model.emb_gru(emb)
    out , _ = dist_model.gru(out)
    out = out[: , -1, :]
    out = dist_model.gru_last(out)
    s1  = out[:batch_size]
    s2  = out[batch_size:]
    l = s1.size(0)//2
    s3  = torch.cat((s1[:l] , s2[:l]) , dim = 0)
    s4  = torch.cat((s1[l:] , s2[l:]) , dim = 0)
    return dist(s1,s2) , dist(s3,s4)
    
    
    

In [0]:

class Video_captioner(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(len(wordtoindex), embedding_dim)
        self.embedding.weight = nn.Parameter(embedding)
        self.embedding.weight.requires_grad = False
        self.lstm1 = nn.LSTM(dim_input1 , dim_hidden1, batch_first = True)
        self.lstm2 = nn.LSTM(dim_input2 , dim_hidden2 , batch_first = True)
        self.img_encode = nn.Linear(dim_image , dim_input1)
        self.embed_word = nn.Linear(dim_hidden2 ,len(wordtoindex))
    def forward(self  , video , objects , feed , teacher = True ):
        emb = self.embedding(feed)
        out  = self.img_encode(video)
        #encoding
        out11 , state11 = self.lstm1(out)
        lstm2_inp = torch.cat( (objects , out11) , dim = 2   )
        out12 , state12 = self.lstm2(lstm2_inp)
        #decoding
        if(teacher == True):
            padding = torch.zeros(video.size(0) , feed.size(1) , dim_input1).to(device).double()
            out21 , state21 = self.lstm1(padding, state11)
            lstm2_inp2 = torch.cat( (emb , out21)  , dim = 2 )
            out22 , state22 = self.lstm2(lstm2_inp2 , state12)
        else:
            padding = torch.zeros(video.size(0) , max_caption_length-1 , dim_input1).to(device).double()
            out21 , state21 = self.lstm1(padding , state11)
            out22 = torch.zeros(video.size(0) , max_caption_length  -1, dim_hidden2).to(device).double()
            state__ = state12
            for i in range(max_caption_length -1):
                inp_lstm2 = torch.cat((emb , out21[: , i:i+1, :]) , dim = 2)
                out__ , state__ = self.lstm2(inp_lstm2 , state__)
                out22[: , i:i+1 , :] = out__
                with torch.no_grad():
                    out__  = self.embed_word(out__)
                    out__ = torch.argmax(out__ , dim = 2)                    
                    out__ = self.embedding(out__)
                    emb = out__
                    
                    
                
                
        out22 = out22.contiguous()    
        soft  = self.embed_word(out22)
        meaning_out = torch.matmul(soft , embedding)
        return soft.view(-1, len(wordtoindex)) , meaning_out 
        
        
        

        
        
        

In [0]:

class Attention_model(nn.Module):
    def __init__(self ):
        super().__init__()
        self.embedding = nn.Embedding(len(wordtoindex), embedding_dim)
        self.embedding.weight = nn.Parameter(embedding)
        self.embedding.weight.requires_grad = False
        self.img_encode = nn.Linear(dim_image , dim_input1)
        self.lstm1 = nn.LSTM(dim_input1 , dim_hidden1, batch_first = True)
        self.lstm2 = nn.LSTM(dim_input2 , dim_hidden2 , batch_first = True)
        self.embed_word = nn.Linear(dim_hidden2 ,len(wordtoindex))
        self.attn = nn.Linear(dim_hidden1  , dim_hidden2)
        self.tanh = nn.Tanh()
        self.context_inp = nn.Linear(dim_hidden1 , dim_hidden1)
    def calculate_attention(self, prev_out , encoder_out):
        seq_len = encoder_out.size(1)
        encoder_out = self.attn(encoder_out)
        encoder_out = encoder_out.transpose(1,2)
        att_energy = torch.bmm(prev_out , encoder_out)
        att_energy =  F.softmax(att_energy , dim = 2)
        return att_energy
    
    def decoder_teacher(self , encoder_out , emb , decoder_state):
        prev_out = torch.zeros(encoder_out.size(0) , 1 , dim_hidden2).to(device).double()
        out22 = torch.zeros( encoder_out.size(0), max_caption_length -1 , dim_hidden2).to(device).double()
        for i in range(max_caption_length - 1):
            attention_energy = self.calculate_attention(prev_out , encoder_out)
            context = torch.bmm(attention_energy , encoder_out)
            context = self.context_inp(context)
            lstm2_inp2 = torch.cat((emb[: , i:i+1 , :] , context ), dim = 2)
            decoder_out , decoder_state = self.lstm2(lstm2_inp2 , decoder_state)
            out22[: , i:i+1 , :] = decoder_out
            prev_out = decoder_out
        return out22
    def decoder_non_teacher(self , encoder_out , emb , decoder_state):
        prev_out = torch.zeros(encoder_out.size(0) , 1 , dim_hidden2).to(device).double()
        out22 = torch.zeros(encoder_out.size(0) , max_caption_length -1 , dim_hidden2).to(device).double()
        for i in range(max_caption_length - 1):
            attention_energy = self.calculate_attention(prev_out , encoder_out)
            context = torch.bmm(attention_energy , encoder_out)
            context = self.context_inp(context)
            lstm2_inp2 = torch.cat((emb , context ), dim = 2)
            decoder_out , decoder_state = self.lstm2(lstm2_inp2 , decoder_state)
            out22[: , i:i+1 , :] = decoder_out
            prev_out = decoder_out
            with torch.no_grad():
                decoder_out  = self.embed_word(decoder_out)
                decoder_out = torch.argmax(decoder_out , dim = 2)                    
                emb = self.embedding(decoder_out)
        return out22
        
            
        
        

    def forward(self  , video , objects , feed , teacher = True):
        emb = self.embedding(feed)
        video  = self.img_encode(video)
        # encode
        out11  , state11 = self.lstm1(video)
        lstm2_inp = torch.cat( (objects , out11) , dim = 2 )
        out12  , decoder_state  = self.lstm2(lstm2_inp)
        encoder_out = out11
        # Decode
        if(teacher == True):
            out22  = self.decoder_teacher(encoder_out , emb , decoder_state)
        else:
            out22 = self.decoder_non_teacher(encoder_out , emb , decoder_state)
            
        out22 = out22.contiguous() 
        soft  = self.embed_word(out22)
        meaning_out = torch.matmul(F.softmax(soft , dim = 2) , embedding)
        return soft.view(-1, len(wordtoindex)) , meaning_out 

In [0]:
#video_model = Video_captioner().to(device).double()
video_model = Attention_model().to(device).double()
video_model.load_state_dict(torch.load(os.path.join(model_save_path3 , "model" + str(370) + ".pt")))
#video_model.lstm1.requires_grad = False
#video_model.lstm2.requires_grad = False
#video_model.attn.requires_grad = False
params_video = list(filter(lambda p: p.requires_grad, video_model.parameters()))
params_sent =  list(filter(lambda p: p.requires_grad, model.parameters()))
optimizer = torch.optim.Adam(params_video, lr)
sent_optimizer = torch.optim.Adam(params_sent, sent_lr)
crit_video = torch.nn.CrossEntropyLoss()
epoochs = 500

In [0]:
def loss1(out , targets):
    targets = targets.contiguous().view(-1)
    return crit_video(out , targets)

def loss2(emb1 , targets):
    emb2  = video_model.embedding(targets)
    emb = torch.cat((emb1 , emb2) , dim = 0)
    dis = distance(model ,emb , emb1.size(0))
    loss_same = torch.exp(-dis[0]).mean()
    loss_diff = torch.exp(-dis[1]).mean()
    return (1 - loss_same) , loss_diff

total_loss_arr   = []
meaning_array = []

In [0]:
bos = torch.tensor(wordtoindex["<bos>"]).to(device)
bos = bos.unsqueeze(0).unsqueeze(0)
bos = bos.expand(batch_size , 1)

In [17]:
bos.shape

torch.Size([50, 1])

In [0]:

def train1(d):
    out = video_model(d[0] , d[1] , d[2] , True)
    l1 = loss1(out[0] , d[3])
    loss = l1 
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return loss.data.item()
def train3(d):
    out = video_model(d[0] , d[1] , bos , False)
    l1 = loss1(out[0] , d[3])
    loss = l1 
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return loss.data.item()

        
            

In [0]:
def train2(d):
    out = video_model(d[0] , d[1] , bos , False)
    l2_same , l2_diff = loss2(out[1] , d[3])
    l2_same.backward( retain_graph=True)
    sent_optimizer.step()
    sent_optimizer.zero_grad()
    optimizer.step()
    optimizer.zero_grad()
    l2_diff.backward()
    sent_optimizer.step()
    sent_optimizer.zero_grad()
    loss  = l2_same + l2_diff
    return loss.data.item()
      

In [0]:
rep = 1200//batch_size
def train():
    for epoch in range(epoochs):
        print(epoch)
        total_loss = 0
        total_meaning = 0
        count1 = 0
        count2 = 0
        for _ in range(rep):
            d = get_batch(df)
            flag = random.random()
            if(flag <= .35 ):
                l1 =  train1(d)
                total_loss = total_loss + l1
                count1 = count1 + 1
            else:
                l2 = train2(d) 
                total_meaning = total_meaning + l2
                count2 = count2 + 1
        if(count1 != 0):
            total_loss = total_loss/count1
        if(count2 != 0):
            total_meaning = total_meaning/count2
        total_loss_arr.append(total_loss)
        meaning_array.append(total_meaning)
        print("teacher loss  count" , total_loss , count1)
        print("non teach loss count", total_meaning , count2)
        if((epoch + 1)%10 == 0):
            torch.save(model.state_dict(), os.path.join(model_sent_teacher , "model" + str(epoch + 1) + ".pt"))
            torch.save(video_model.state_dict(), os.path.join(model_save_teacher , "model" + str(epoch + 1) + ".pt"))
            print("model saved")


In [0]:
train()

0
teacher loss  count 0.7110329452850973 12
non teach loss count 0.38573355422597705 12
1
teacher loss  count 0.7171902356825006 8
non teach loss count 0.3078689324031274 16
2
teacher loss  count 0.6535796231250711 7
non teach loss count 0.28292197023754817 17
3
teacher loss  count 0.6595703306457105 8
non teach loss count 0.2597542042192105 16
4
teacher loss  count 0.695228827657072 10
non teach loss count 0.24125750209489696 14
5
teacher loss  count 0.7734968959252618 1
non teach loss count 0.2367421629152599 23
6
teacher loss  count 0.6027518915871376 6
non teach loss count 0.22778151021664395 18
7
teacher loss  count 0.7290023228706628 5
non teach loss count 0.23034795523893178 19
8
teacher loss  count 0.7742158298067789 9
non teach loss count 0.19821918345050482 15
9
teacher loss  count 0.6683137946318228 5
non teach loss count 0.19564014038210664 19
model saved
10
teacher loss  count 0.7341490331192466 5
non teach loss count 0.19727749307646003 19
11
teacher loss  count 0.6864289